In [30]:
from bs4 import BeautifulSoup

import requests
import os
from requests.exceptions import MissingSchema

In [31]:
def get_site_text(url):

    try:
        response = requests.get(url)
    except MissingSchema:
        return ""

    html_content = response.text
    soup = BeautifulSoup(html_content, 'html.parser')

    return soup.get_text()

In [32]:
def get_links(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    urls = [url]
    for link in soup.find_all('a'):
        if link not in urls:
                urls.append(link.get('href'))

    return urls

In [33]:
url = "https://aardwolfarchers.org.uk"
filename = "aardwol.txt"

In [34]:
list_of_urls = get_links(url)
print(list_of_urls)

['https://aardwolfarchers.org.uk', '#main', 'https://aardwolfarchers.org.uk/', 'https://aardwolfarchers.org.uk/', 'https://aardwolfarchers.org.uk/about/', 'https://aardwolfarchers.org.uk/course/', 'https://aardwolfarchers.org.uk/books/', 'https://aardwolfarchers.org.uk/contact/', 'https://aardwolfarchers.org.uk/resources/', 'https://aardwolfarchers.org.uk/lessons/', 'https://aardwolfarchers.org.uk/', 'https://aardwolfarchers.org.uk/course/', 'https://pindersschoolwear.com/', 'https://aardwolfarchers.org.uk/about/', 'https://aardwolfarchers.org.uk/course/', 'https://aardwolfarchers.org.uk/books/', 'https://aardwolfarchers.org.uk/contact/', 'https://aardwolfarchers.org.uk/lessons/', 'https://www.kadencewp.com/', 'https://aardwolfarchers.org.uk/', 'https://aardwolfarchers.org.uk/about/', 'https://aardwolfarchers.org.uk/course/', 'https://aardwolfarchers.org.uk/books/', 'https://aardwolfarchers.org.uk/contact/', 'https://aardwolfarchers.org.uk/resources/', 'https://aardwolfarchers.org.uk/l

In [35]:
def format_string(string):
    string = string.strip()
    string = os.linesep.join([s for s in string.splitlines() if s])
    return string

In [36]:
site_text = ""
with open(filename, "w") as file:
    for site in list_of_urls:
        formatted_string = format_string(get_site_text(site) + "\n\n")
        file.write(formatted_string)
